In [41]:
#Import modules
import os
import pandas as pd
import pybedtools
import re

In [77]:
#Create a bedtools of the relevant regions and append their score at the end of it
def get_regions(feature_scores_table, data_table):
    feature_scores = pd.read_csv(feature_scores_table, index_col=0)
    data = pd.read_table(data_table)
    
    
    cv_regions = pybedtools.BedTool(
        [list(data.loc[index, ['Chromosome', 'Start', 'End']]) + [str(row[0])]
         for index, row in feature_scores.iterrows()]
    )
    return cv_regions


xg_cv_regions = get_regions('../data/XGBoost_scores.csv', '../data/Train_call.txt')
mi_cv_regions = get_regions('../data/MI_scores.csv', '../data/Train_call.txt')

In [56]:
#Create a bed tools of the relevant genes
cancer_genes_table = pd.read_csv('../data/census_cancer_genes.csv')
cancer_genes = pybedtools.BedTool(
    [re.split(r'[:\-]', row['Genome Location']) + [row['Gene Symbol']]
    for i, row in cancer_genes_table.iterrows()
         if ':-' not in row['Genome Location']
])

In [108]:
#Intersect with the regions to identify relevant genes
def intersect(regions, genes):
    relevant_gene_symbols = [gene.fields[3] for gene in genes.intersect(regions, wb=True)]
    relevant_regions = [f'{gene.fields[4]}:{gene.fields[5]}-{gene.fields[6]}' for gene in genes.intersect(regions, wb=True)]
    relevant_region_scores = [gene.fields[7] for gene in genes.intersect(regions, wb=True)]
    mapk = [gene.fields for gene in genes.intersect(regions, wb=True) if gene.fields[3] == 'MAPK1']
    print(mapk)
#     import collections
#     print([item for item, count in collections.Counter(relevant_gene_symbols).items() if count > 1])
    
#     relevant_genes_table = cancer_genes_table[cancer_genes_table['Gene Symbol'].isin(relevant_gene_symbols)]
#     relevant_genes_table['CV Region'] = relevant_regions
#     relevant_genes_table['CV Region Score'] = relevant_region_scores
#     relevant_genes_table.sort_values(by='CV Region Score', inplace=True, ascending=False)
 
    return relevant_genes_table

In [84]:
#Intersect with XGBoost regions
display(intersect(xg_cv_regions, cancer_genes))

/var/folders/n_/bxvsx0617pz99z9_15ddjbpw0000gq/T/ipykernel_15442/662973930.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_genes_table['CV Region'] = relevant_regions
/var/folders/n_/bxvsx0617pz99z9_15ddjbpw0000gq/T/ipykernel_15442/662973930.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_genes_table['CV Region Score'] = relevant_region_scores
/var/folders/n_/bxvsx0617pz99z9_15ddjbpw0000gq/T/ipykernel_15442/662973930.py:10: SettingWithCopyWarning: 
A value is trying to be set on a 

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),...,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms,CV Region,CV Region Score
25,ARHGAP26,Rho GTPase activating protein 26,23092.0,5:142770377-143229011,1,Yes,31.30,yes,NaN,"AML, MDS",...,L,Dom,"TSG, fusion","T, F, S",KMT2A,NaN,NaN,"23092,ARHGAP26,ENSG00000145819.16,GRAF,KIAA062...",5:142831583-149027600,3.7971980571746826
707,WIF1,WNT inhibitory factor 1,11197.0,12:65050626-65121566,1,Yes,14.30,yes,NaN,pleomorphic salivary gland adenoma,...,E,Dom,"TSG, fusion",T,HMGA2,NaN,NaN,"11197,ENSG00000156076.9,Q9Y5W5,WIF1",12:64727853-66012212,1.9669854640960691
300,HMGA2,high mobility group AT-hook 2 (HMGIC),8091.0,12:65824460-65915447,1,NaN,14.30,yes,NaN,"lipoma, leiomyoma, pleomorphic salivary gland ...",...,M,Dom,"oncogene, fusion",T,"LHFP, RAD51B, LPP, COX6C, ACKR3, NFIB, ALDH2, ...",NaN,NaN,"8091,BABL,ENSG00000149948.13,HMGA2,HMGIC,LIPO",12:64727853-66012212,1.9669854640960691
612,SMAD4,SMAD family member 4,4089.0,18:51030213-51085039,1,Yes,21.20,yes,yes,"colorectal, pancreatic, small intestine",...,E,Rec,TSG,"D, Mis, N, F",NaN,NaN,NaN,"4089,DPC4,ENSG00000141646.13,MADH4,Q13485,SMAD4",18:48187829-51613228,1.4211688041687012
548,RAD17,RAD17 checkpoint clamp loader component,5884.0,5:69369806-69414801,2,NaN,13.20,yes,NaN,"lymphoma, colon cancer",...,"E, L",NaN,TSG,D,NaN,NaN,NaN,"5884,CCYC,ENSG00000152942.18,O75943,RAD17,RAD1...",5:69274433-70345552,1.3008345365524292
539,PTPRB,"protein tyrosine phosphatase, receptor type, B",5787.0,12:70515866-70637440,1,Yes,15.00,yes,NaN,angiosarcoma,...,E,Rec,TSG,"N, Mis, S, F",NaN,NaN,NaN,"5787,ENSG00000127329.15,P23467,PTPB,PTPRB",12:70462057-70560603,1.2548660039901731
272,FUS,"fusion, derived from t(12;16) malignant liposa...",2521.0,16:31180110-31194871,1,Yes,11.20,yes,NaN,"liposarcoma, AML, Ewing sarcoma, angiomatoid f...",...,"M, L",Dom,"TSG, fusion",T,"DDIT3, ERG, FEV, ATF1, CREB3L2, CREB3L1, NFATC2",NaN,NaN,"2521,ALS6,ENSG00000089280.18,FUS,FUS1,HNRNPP2,...",16:30561643-31835555,1.029874324798584
47,B2M,beta-2-microglobulin,567.0,15:44711477-44718877,1,Yes,21.10,yes,NaN,"DLBCL, melanoma, colorectal adenocarcinoma",...,"E, L",Rec,TSG,"Mis, N, F",NaN,yes,Immunodeficiency 43,"567,B2M,ENSG00000166710.17,P61769",15:44193179-48393597,0.95516699552536
301,HMGN2P46,high mobility group nucleosomal binding domain...,NaN,15:45511136-45556730,2,NaN,21.10,yes,NaN,prostate,...,E,Dom,fusion,T,ETV1,NaN,NaN,NaN,15:44193179-48393597,0.95516699552536
588,SDHD,"succinate dehydrogenase complex, subunit D, in...",6392.0,11:112086773-112095794,1,NaN,23.10,NaN,yes,NaN,...,O,Rec,TSG,"Mis, N, F, S",NaN,NaN,NaN,"6392,ENSG00000204370.10,O14521,PGL,PGL1,SDHD,cybS",11:109089452-114667900,0.8724284172058105


In [111]:
#Intersect with MI regions
inter = intersect(mi_cv_regions, cancer_genes)
display(cancer_genes_table[cancer_genes_table['Gene Symbol'] == 'MAPK1'])

[['22', '21754500', '21844483', 'MAPK1', '22', '20906896', '21844483', '0.1104099727041183'], ['22', '21858001', '21867629', 'MAPK1', '22', '21858001', '22597688', '0.0797304208405578']]


,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms
393,MAPK1,mitogen-activated protein kinase 1,5594.0,22:21754500-21867629,1,Yes,11.22,yes,NaN,"CLL, ovarian mixed germ cell tumour, cervical ...",NaN,NaN,"E, L",Dom,oncogene,Mis,NaN,NaN,NaN,"5594,ENSG00000100030.14,ERK,ERK2,MAPK1,MAPK2,P..."
